A very good tutorial:
https://annefou.github.io/metos_python/02-formats/

http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/11_create_netcdf_python.pdf


# CF-Convention Compliance Checker for NetCDF Format
http://puma.nerc.ac.uk/cgi-bin/cf-checker.pl

# CF-Convention book
http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.pdf

# NetCDF4 example
http://unidata.github.io/netcdf4-python/

https://github.com/Unidata/netcdf4-python/blob/master/examples/tutorial.py

# Grid Mappings and Projections

http://cfconventions.org/Data/cf-conventions/cf-conventions-1.0/build/ch05s06.html

http://cfconventions.org/Data/cf-conventions/cf-conventions-1.0/build/apf.html

http://nsidc.org/data/atlas/laea_north_nsidc.html

http://geotiff.maptools.org/proj_list/lambert_azimuthal_equal_area.html


# PROJ.4 Organization

  +proj=laea  
  
  +lat_0=Latitude at projection center 
              
  +lon_0=Longitude at projection center
  
  +x_0=False Easting
  
  +y_0=False Northing
              

In [32]:
from osgeo import ogr, osr, gdal
import numpy as np
import netCDF4
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num, date2index
import pyproj

source_path = 'UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif'
#source_path = "outfile.tif"
output_path = '2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001_test_convtif.nc'

source_dataset = gdal.Open(source_path)
gdal.Translate(output_path, source_dataset, format='NetCDF', )
converted_dataset = netCDF4.Dataset(output_path, 'r+')
converted_dataset.createDimension('time', None)
times = converted_dataset.createVariable('time', 'i4', ('time', ))
times.units = 'hours since 1990-01-01 00:00:00'
times.calendar = 'gregorian'
times.long_name = 'time'
times.axis = 'T'
times[:] = date2num([datetime(2008, 10, 1)],units=times.units,calendar=times.calendar)

band1 = converted_dataset.variables['Band1']
print(band1)
#grid_mapping = converted_dataset.variables['lambert_azimuthal_equal_area']
#print(grid_mapping)
lst = converted_dataset.createVariable('LST_LWST_avg_daily', band1.datatype, ('time', 'y', 'x'))
lst.setncatts({k: band1.getncattr(k) for k in band1.ncattrs()})
lst.units = 'K'
lst.long_name = 'average daily temperature'
np.warnings.filterwarnings('ignore')
lst[:] = [band1[:]]
print(lst)
print(type(band1[:]))
print(lst[:])


# Construct the grid
meta = ds.GetMetadata()
x0, xinc, _, y0, _, yinc = ds.GetGeoTransform()
nx, ny = (ds.RasterXSize, ds.RasterYSize)
x = np.linspace(x0, x0 + xinc*nx, nx)
y = np.linspace(y0, y0 + yinc*ny, ny)
xv, yv = np.meshgrid(x, y)

# Reproject the coordinates out of lamaz into lat/lon.
lamaz = pyproj.Proj("+proj=laea +lat_0=90 +lon_0=0 k=1 +x_0=0 +y_0= 0 +units=m +a=6371228.0 +b=6371228.0")
wgs84 = pyproj.Proj("+proj=longlat") 
lon, lat= pyproj.transform(lamaz, wgs84, xv, yv)


# x = np.array(converted_dataset.variables['x'][:])
# y = np.array(converted_dataset.variables['y'][:])
# xv, yv = np.meshgrid(x, y)
# lamaz = pyproj.Proj("+proj=laea +lat_0=90 +lon_0=0 +units=m")
# wgs84 = pyproj.Proj("+init=EPSG:4326") 
# lon, lat= pyproj.transform(lamaz, wgs84, xv, yv)

var_lat = converted_dataset.createVariable('lat', 'f4', ('y','x'))
var_lat.units = "degree" 
var_lat.long_name = 'Latitude. -90 to 90 deg.'
    
var_lon = converted_dataset.createVariable('lon', 'f4', ('y','x'))
var_lon.units = "degree"
var_lon.long_name = 'Longitude. -180 to 180 deg.'
var_lon[:,:] = lon[:,:]
var_lat[:,:] = lat[:,:]

converted_dataset.close()

<class 'netCDF4._netCDF4.Variable'>
float32 Band1(y, x)
    long_name: GDAL Band Number 1
    _FillValue: 9.96921e+36
    grid_mapping: lambert_azimuthal_equal_area
unlimited dimensions: 
current shape = (1956, 1299)
filling off

<class 'netCDF4._netCDF4.Variable'>
float32 LST_LWST_avg_daily(time, y, x)
    grid_mapping: lambert_azimuthal_equal_area
    long_name: average daily temperature
    _FillValue: 9.96921e+36
    units: K
unlimited dimensions: time
current shape = (1, 1956, 1299)
filling off

<class 'numpy.ndarray'>
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]


In [6]:
gdal.Info("UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif").split('\n')

['Driver: GTiff/GeoTIFF',
 'Files: UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif',
 'Size is 1299, 1956',
 'Coordinate System is:',
 'PROJCS["unnamed",',
 '    GEOGCS["unknown",',
 '        DATUM["unknown",',
 '            SPHEROID["unnamed",6371228,0]],',
 '        PRIMEM["Greenwich",0],',
 '        UNIT["degree",0.0174532925199433]],',
 '    PROJECTION["Lambert_Azimuthal_Equal_Area"],',
 '    PARAMETER["latitude_of_center",90],',
 '    PARAMETER["longitude_of_center",0],',
 '    PARAMETER["false_easting",0],',
 '    PARAMETER["false_northing",0],',
 '    UNIT["metre",1,',
 '        AUTHORITY["EPSG","9001"]]]',
 'Origin = (-5386000.000000000931323,470999.999999999534339)',
 'Pixel Size = (1000.000000000000000,-1000.000000000000000)',
 'Metadata:',
 '  AREA_OR_POINT=Area',
 '  TIFFTAG_DATETIME=2018:02:13 13:18:46',
 '  TIFFTAG_DOCUMENTNAME=C:\\Users\\h2kheyro\\Documents\\Data_Analysis\\Great Lakes\\LST\\MonthlyAve2/UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.

In [26]:
# http://hdfeos.org/zoo/NSIDC/MYD29P1D_A2010133_h11v05_005_2010135032246_1km_Sea_Ice_by_Refl.py

ds = gdal.Open("UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif")
data = ds.ReadAsArray()

# Construct the grid
meta = ds.GetMetadata()
x0, xinc, _, y0, _, yinc = ds.GetGeoTransform()
nx, ny = (ds.RasterXSize, ds.RasterYSize)
x = np.linspace(x0, x0 + xinc*nx, nx)
y = np.linspace(y0, y0 + yinc*ny, ny)
xv, yv = np.meshgrid(x, y)

# Reproject the coordinates out of lamaz into lat/lon.
lamaz = pyproj.Proj("+proj=laea +a=6371228 +lat_0=90 +lon_0=0 +units=m")
wgs84 = pyproj.Proj("+init=EPSG:4326") 
lon, lat= pyproj.transform(lamaz, wgs84, xv, yv)


array([[39.78795855, 39.79785935, 39.80775974, ..., 52.30895897,
        52.31840596, 52.32785266],
       [39.78882324, 39.79872417, 39.80862469, ..., 52.31004574,
        52.31949296, 52.32893989],
       [39.78968611, 39.79958716, 39.8094878 , ..., 52.31113022,
        52.32057767, 52.33002483],
       ...,
       [37.99487439, 38.00452829, 38.01418167, ..., 50.08184865,
        50.09084961, 50.09985005],
       [37.99221602, 38.00186957, 38.0115226 , ..., 50.07858395,
        50.08758429, 50.09658412],
       [37.98955595, 37.99920915, 38.00886183, ..., 50.07531726,
        50.08431699, 50.09331621]])

In [8]:
gdal.Info("outfile.tif").split("\n")

['Driver: GTiff/GeoTIFF',
 'Files: outfile.tif',
 'Size is 2307, 1269',
 'Coordinate System is:',
 'GEOGCS["WGS 84",',
 '    DATUM["unknown",',
 '        SPHEROID["WGS84",6378137,298.257223563]],',
 '    PRIMEM["Greenwich",0],',
 '    UNIT["degree",0.0174532925199433]]',
 'Origin = (-96.573962888870696,52.584333014393735)',
 'Pixel Size = (0.011502944238810,-0.011502944238810)',
 'Metadata:',
 '  AREA_OR_POINT=Area',
 '  TIFFTAG_DATETIME=2018:02:13 13:18:46',
 '  TIFFTAG_DOCUMENTNAME=C:\\Users\\h2kheyro\\Documents\\Data_Analysis\\Great Lakes\\LST\\MonthlyAve2/UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif',
 '  TIFFTAG_IMAGEDESCRIPTION=IDL TIFF file',
 '  TIFFTAG_RESOLUTIONUNIT=2 (pixels/inch)',
 '  TIFFTAG_SOFTWARE=IDL 7.1.2, ITT Visual Information Solutions',
 '  TIFFTAG_XRESOLUTION=100',
 '  TIFFTAG_YRESOLUTION=100',
 'Image Structure Metadata:',
 '  INTERLEAVE=BAND',
 'Corner Coordinates:',
 'Upper Left  ( -96.5739629,  52.5843330) ( 96d34\'26.27"W, 52d35\' 3.60

In [29]:
ds_example = netCDF4.Dataset("/home/l8tan/Downloads/air.departure.sig995.2012.nc")
ds_example.variables["air_dep"][:]

array([[[-68.17999268, -68.17999268, -68.17999268, ..., -68.17999268,
         -68.17999268, -68.17999268],
        [-66.97998047, -66.92999268, -66.87997437, ..., -66.79998779,
         -66.87997437, -66.94998169],
        [-63.87997437, -63.07998657, -62.19998169, ..., -65.47998047,
         -65.08999634, -64.52999878],
        ..., 
        [-50.72998047, -50.57998657, -50.42999268, ..., -50.97998047,
         -50.92999268, -50.87997437],
        [-51.57998657, -51.54998779, -51.5       , ..., -51.62997437,
         -51.60998535, -51.60998535],
        [-52.62997437, -52.62997437, -52.62997437, ..., -52.62997437,
         -52.62997437, -52.62997437]],

       [[-68.04000854, -68.04000854, -68.04000854, ..., -68.04000854,
         -68.04000854, -68.04000854],
        [-70.92999268, -71.07998657, -71.20001221, ..., -70.48001099,
         -70.63000488, -70.79000854],
        [-62.07998657, -61.3500061 , -60.67999268, ..., -64.02999878,
         -63.42999268, -62.75      ],
        ...,

In [2]:
int(gdal.VersionInfo('VERSION_NUM'))

2020200

In [6]:
ds = gdal.Open('UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif')

In [17]:
ds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f7cfea5e1e0> >

In [3]:
help(gdal.Translate)

Help on function Translate in module osgeo.gdal:

Translate(destName, srcDS, **kwargs)
    Convert a dataset.
    Arguments are :
      destName --- Output dataset name
      srcDS --- a Dataset object or a filename
    Keyword arguments are :
      options --- return of gdal.TranslateOptions(), string or array of strings
      other keywords arguments of gdal.TranslateOptions()
    If options is provided as a gdal.TranslateOptions() object, other keywords are ignored.



In [7]:
gdal.Translate('test_2.nc', ds, format='NetCDF')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7ffba9be2630> >

In [4]:
help(gdal.TranslateOptions)

Help on function TranslateOptions in module osgeo.gdal:

TranslateOptions(options=[], format='GTiff', outputType=0, bandList=None, maskBand=None, width=0, height=0, widthPct=0.0, heightPct=0.0, xRes=0.0, yRes=0.0, creationOptions=None, srcWin=None, projWin=None, projWinSRS=None, strict=False, unscale=False, scaleParams=None, exponents=None, outputBounds=None, metadataOptions=None, outputSRS=None, GCPs=None, noData=None, rgbExpand=None, stats=False, rat=True, resampleAlg=None, callback=None, callback_data=None)
    Create a TranslateOptions() object that can be passed to gdal.Translate()
    Keyword arguments are :
      options --- can be be an array of strings, a string or let empty and filled from other keywords.
      format --- output format ("GTiff", etc...)
      outputType --- output type (gdal.GDT_Byte, etc...)
      bandList --- array of band numbers (index start at 1)
      maskBand --- mask band to generate or not ("none", "auto", "mask", 1, ...)
      width --- width of the

In [27]:
import netCDF4
f = netCDF4.Dataset('test.nc', 'r+')
print(f)
#f.createDimension('time', None)
#f.renameDimension('x', 'rlon')
#f.renameDimension('y', 'rlat')
# f.renameVariable('x', 'lon')
# f.renameVariable('y', 'lat')
# f.renameVariable('Band1', 'CaLDAS_*')
#f.renameVariable('lambert_azimuthal_equal_area', 'rotated_pole')
times = f.createVariable('time', float, ('time', ))
times.units = "days since 0001-01-01 00:00:00"
times.calendar = 'gregorian'
print(f)
f.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    GDAL_AREA_OR_POINT: Area
    GDAL_TIFFTAG_DATETIME: 2018:02:13 13:18:46
    GDAL_TIFFTAG_DOCUMENTNAME: C:\Users\h2kheyro\Documents\Data_Analysis\Great Lakes\LST\MonthlyAve2/UW_NA_LST_7.1.1_001_2008.10.1_0.0.0_2008.10.31_23.59.59_NA_001.tif
    GDAL_TIFFTAG_IMAGEDESCRIPTION: IDL TIFF file
    GDAL_TIFFTAG_RESOLUTIONUNIT: 2 (pixels/inch)
    GDAL_TIFFTAG_SOFTWARE: IDL 7.1.2, ITT Visual Information Solutions
    GDAL_TIFFTAG_XRESOLUTION: 100
    GDAL_TIFFTAG_YRESOLUTION: 100
    Conventions: CF-1.5
    GDAL: GDAL 1.11.3, released 2015/09/16
    history: Mon Mar 05 16:27:26 2018: GDAL CreateCopy( test.nc, ... )
    dimensions(sizes): rlon(1299), rlat(1956), time(0)
    variables(dimensions): |S1 rotated_pole(), float64 lon(rlon), float64 lat(rlat), float32 CaLDAS_*(rlat,rlon)
    groups: 

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    G

In [18]:
a = ds.ReadAsArray()
np.shape(a)

(1956, 1299)

In [6]:
ds.GetGeoTransform()

(-5386000.000000001, 1000.0, 0.0, 470999.99999999953, 0.0, -1000.0)

In [7]:
gdal.GetDriverByName("GTiff")

<osgeo.gdal.Driver; proxy of <Swig Object of type 'GDALDriverShadow *' at 0x7f7d3c20ca80> >

In [11]:
help(gdal.Open)

Help on function Open in module osgeo.gdal:

Open(*args)
    Open(char const * utf8_path, GDALAccess eAccess) -> Dataset



In [12]:
help(gdal.Translate)

AttributeError: module 'osgeo.gdal' has no attribute 'Translate'

https://stackoverflow.com/questions/33807774/package-a-timeseries-of-geotiff-rasters-as-a-single-netcdf-file

https://stackoverflow.com/questions/46899337/convert-raster-time-series-of-multiple-geotiff-images-to-netcdf


https://gis.stackexchange.com/questions/176990/write-stacks-of-geotiff-image-to-netcdf-by-chunk-size

https://gis.stackexchange.com/questions/70458/convert-timeseries-stack-of-gtiff-raster-to-single-netcdf

In [ ]:
import numpy as np
import datetime as dt
import os
import gdal
import netCDF4
import re

ds = gdal.Open('/data_sample/R20060111_075725___SIG0__ASAWS___M1VVD_TUW_SGRT15A00_AF075M_E060N054T6.tif')

a = ds.ReadAsArray()
nlat, nlon = np.shape(a)

b = ds.GetGeoTransform()  # bbox, interval
lon = np.arange(nlon)*b[1]+b[0]
lat = np.arange(nlat)*b[5]+b[3]

basedate = dt.datetime(2006, 01, 11, 0, 0, 0)

# Create NetCDF file
nco = netCDF4.Dataset('time_series.nc', 'w', clobber=True)

chunk_lon = 16
chunk_lat = 16
chunk_time = 12

# Create dimensions, variables and attributes:
nco.createDimension('lon', nlon)
nco.createDimension('lat', nlat)
nco.createDimension('time', None)

timeo = nco.createVariable('time', 'f4', ('time',))
timeo.units = 'days since 2006-01-11 00:00:00'
timeo.standard_name = 'time'

lono = nco.createVariable('lon', 'f4', ('lon',))
lono.standard_name = 'longitude'

lato = nco.createVariable('lat', 'f4', ('lat',))
lato.standard_name = 'latitude'

# Create container variable for CRS: lon/lat WGS84 datum
crso = nco.createVariable('crs', 'i4')
crso.long_name = 'Lon/Lat Coords in WGS84'
crso.grid_mapping_name = 'latitude_longitude'
crso.longitude_of_prime_meridian = 0.0
crso.semi_major_axis = 6378137.0
crso.inverse_flattening = 298.257223563

# Create short integer variable for temperature data, with chunking
tmno = nco.createVariable('tmn', 'i2',  ('time', 'lat', 'lon'),
                          zlib=True, chunksizes=[chunk_time,
                                                 chunk_lat,
                                                 chunk_lon],
                          fill_value=-9999)
tmno.units = 'degC'
tmno.scale_factor = 0.01
tmno.add_offset = 0.00
tmno.long_name = 'minimum monthly temperature'
tmno.standard_name = 'air_temperature'
tmno.grid_mapping = 'crs'
tmno.set_auto_maskandscale(False)

nco.Conventions = 'CF-1.6'

# Write lon,lat
lono[:] = lon
lato[:] = lat

pat = re.compile('(\w{1})(\d{4})(\d{2})(\d{2})_\d{6}')
itime = 0

# Step through data, writing time and data to NetCDF
for root, dirs, files in os.walk('/data_sample/'):
    dirs.sort()
    files.sort()
    for f in files:
        if re.match(pat, f):
            # read the time values by parsing the filename
            year = int(f[4:8])
            mon = int(f[8:10])
            date = dt.datetime(year, mon, 1, 0, 0, 0)
            print(date)
            dtime = (date-basedate).total_seconds()/86400.
            timeo[itime] = dtime
            # min temp
            tmn_path = os.path.join(root, f)
            print(tmn_path)
            tmn = gdal.Open(tmn_path)
            a = tmn.ReadAsArray()  # data
            tmno[itime, :, :] = a
            itime = itime+1

nco.close()

In [1]:
import gdal

In [4]:
help(gdal.TranslateOption)

AttributeError: module 'gdal' has no attribute 'Translate'